# Bootstrap Development and Validation

In [ ]:
source("notebooks/initialize-data-analysis.r", local = TRUE)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Rows: 5669 Columns: 113
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr   (4): patient_ID, site, dept, LZD_route
dbl  (20): patient_age, patient_weight, charlson, baseline_CLCR, baseline_WB...
lgl  (85): patient_sex, dept_ICU, dept_ER, dept_other, invasive_ETI, invasiv...
date  (4): baseline_date, LZD_start, LZD_end, test_date

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column t

In [ ]:
library(BAS)
library(rsample) # for bootstraps()
library(furrr) # for future_map()

Loading required package: future

── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.5     ✔ recipes      1.0.9
✔ dials        1.2.0     ✔ tune         1.1.2
✔ infer        1.0.5     ✔ workflows    1.1.3
✔ modeldata    1.3.0     ✔ workflowsets 1.0.1
✔ parsnip      1.1.1     ✔ yardstick    1.3.0

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks readr::spec()
✖ recipes::step()   masks stats::step()
• Use suppressPackageStartupMessages() to eliminate package startup messages

Loading required package: rms

Loading required package: Hmisc


Attaching package: 'Hmisc'

The following object is masked from 'package:parsnip':

    translate

The following objects are masked from 'package:dplyr':

    src, summarize

The following objects are masked from 'package:base':

    format.pval, units

In [ ]:
load("data/model-performance/model_full.rda")

pHat <- predict(model_full$fit, data_patient_complete, type = "response")
yTest <- data_patient_complete$flag_ADR_TP_ID
calperf <- valProbggplot(pHat, yTest)

apparent_performance_metrics <- tibble(
  apparent_C_index = calperf$Cindex[[1]],
  apparent_calibration_intercept = calperf$Calibration$Intercept[[1]],
  apparent_calibration_slope = calperf$Calibration$Slope[[1]]
)

corrected_performance_metrics <- bind_cols(apparent_performance_metrics, optimism_estimates) |>
  mutate(
    corrected_C_index = apparent_C_index - mean_optimism_C_index,
    corrected_calibration_intercept = apparent_calibration_intercept - mean_optimism_calibration_intercept,
    corrected_calibration_slope = apparent_calibration_slope - mean_optimism_calibration_slope
  ) |>
  select(starts_with("corrected"))

In [ ]:
save(corrected_performance_metrics, file = "data/model-performance/corrected_performance_metrics.rda")